# Instruction Finetuning


### 데이터셋 구축

1. 목적 정의: 먼저, 세부 튜닝을 통해 달성하고자 하는 목표를 명확히 합니다.
1. 데이터 수집: 목표에 맞는 데이터를 수집합니다. 이 데이터는 공개 데이터셋일 수도 있고, 사용자가 직접 수집한 데이터일 수도 있습니다.

1. 데이터 가공: 수집한 데이터를 모델 훈련에 적합하게 가공합니다. 이 과정에서는 데이터를 정제하고, 필요한 형식으로 변환하는 작업이 포함됩니다.

### 공개 데이터셋 다운로드

In [ ]:
!pip install -U datasets==2.17.0

In [ ]:
from datasets import load_dataset

# 데이터셋 로드
dataset = load_dataset("royboy0416/ko-alpaca")

# 데이터셋의 구조 확인
print(dataset)

In [ ]:
dataset['train'][0]

# Gemma 데이터셋 포맷팅

```<start_of_turn>user```<br>
```What is Cramer's Rule?<end_of_turn>```<br>
```<start_of_turn>model```<br>
```Cramer's Rule is ...<end_of_turn>```

In [ ]:
# 'prompt' 필드 생성 함수
def format_instruction(example):

    # 추가 컨텍스트(input 필드)가 있는 경우
    if example['input'] and len(example['input']) > 0:
        text = f"""<start_of_turn>user\n{example["instruction"]}\n{example["input"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""
    # input 필드가 없는 경우
    else:
        text = f"""<start_of_turn>user\n{example["instruction"]}<end_of_turn>\n<start_of_turn>model\n{example["output"]}<end_of_turn>"""

    return {'prompt': text}

# 데이터셋의 prompt 필드를 업데이트
dataset = dataset.map(format_instruction)

In [ ]:
dataset['train'][0]

In [ ]:
dataset['train'][5]

### 모델 로드 및 튜닝:

1. 모델 학습: gemma-2b 모델을 로드하고, 준비된 데이터셋을 사용하여 모델을 세부 튜닝합니다. 이 과정에서는 학습률, 에폭 수 등의 파라미터를 조정할 수 있습니다.
1. 평가 및 반복: 튜닝된 모델을 평가하고 결과를 확인합니다. 필요에 따라 여러 번 반복하여 모델의 성능을 최적화할 수 있습니다.

In [ ]:
!pip install -qU transformers==4.38.0 accelerate==0.27.1 bitsandbytes==0.42.0 peft==0.8.2 trl==0.7.10

In [ ]:
!nvidia-smi

In [ ]:
import torch
import pandas as pd
import numpy as np
import warnings
import json
import time

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})

tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

In [ ]:
train_data = dataset["train"]
test_data = dataset["test"]

In [ ]:
print(train_data[0])

In [ ]:
def get_completion(query: str, model, tokenizer):

  prompt_template = """<start_of_turn>user
  {query}
  <end_of_turn>
  <start_of_turn>model
  """
  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to("cuda:0")
  generated_ids = model.generate(**model_inputs, max_new_tokens=256)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded

# Fine tuning 이전
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.", model=model, tokenizer=tokenizer)
print(result)


In [ ]:
torch.cuda.empty_cache()

lora_config = LoraConfig(
    r=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit",
    ),
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

In [ ]:
# Fine tuning 이후
result = get_completion(query="건강을 유지하기 위한 세 가지 팁을 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

In [ ]:
# Fine tuning 이후
result = get_completion(query="불면증을 해결하는 방법을 세 가지 알려주세요.",
                        model=trainer.model,
                        tokenizer=tokenizer)
print(result)

# 모델 저장

In [ ]:
new_model = "gemma-2b-it-koalpaca-finetuned"
trainer.model.save_pretrained(new_model)